# API Setup 
## Currency Conversion  

Costa Rica

Belinda Brown, belindabrownr04@gmail.com


Jan, 2021

## Installing packages

### 1. Uvicorn 
Is an **ASGI web server implementation for Python**. **ASGI (Asynchronous Server Gateway Interface)** is a spiritual successor to WSGI, intended to provide a standard interface between async-capable Python web servers, frameworks, and applications.
Where WSGI provided a standard for synchronous Python apps, **ASGI provides one for both asynchronous and synchronous apps**, with a WSGI backwards-compatibility implementation and multiple servers and application frameworks 

- host: [default: 127.0.0.1]
- port: [default: 8000]

> > -- <cite> Unicorn [8]  </cite>

In [ ]:
!pip install uvicorn==0.16.0

### 2. FastAPI

FastAPI is a modern, fast (high-performance), web framework for building APIs with Python 3.6+ based on standard Python type hints.

- You will also need an ASGI server, for production such as **Uvicorn or Hypercorn**.

> > -- <cite> FastAPI [9, 10]  </cite>

In [ ]:
!pip install fastapi

In [ ]:
!pip install fastapi-utils

## Import packages 

In [1]:
import os
import glob
import logging
import sys
import openpyxl
from tqdm import tqdm
from datetime import date
import time
import prettytable
from fastapi import Depends, FastAPI
from fastapi_utils.cbv import cbv
from fastapi_utils.inferring_router import InferringRouter
import requests
from tkinter import *
import tkinter as tk
from tkinter import ttk  
import pandas as pd
import numpy as np

## Functions Definitions

**Preprocessing data**

In [2]:
def preprocessing_data(filename, column_name, column_id):            
    source_df = pd.read_csv(filename, sep=',', encoding='utf-8', engine='python',error_bad_lines=False) 

    source_df[column_name] = [str(i).replace(" ", "") for i in source_df[column_name]]
    source_df = source_df.astype({column_name:'float64'})
    source_df = source_df.drop_duplicates(subset=column_id)   
    curr_df = source_df.copy()
    return curr_df

**Apply to each definition**

In [3]:
def apply_by_row(exchange_df, curr_column_0, new_curr, amount_column, std_amount_column):
    exchange_df[std_amount_column] = exchange_df.apply(lambda row : converter.convert(row[curr_column_0], new_curr, row[amount_column]), axis = 1)
    return exchange_df

**Export to csv definition**

In [4]:
def df_export_to_csv(source_df, output_file_name_with_path):
    source_df.to_csv(output_file_name_with_path, index=False)    

**FastAPI()** carries: 
- The endpoint 
- HTTP method
- Input arguments
- What the API will do behind the scenes.

> > -- <cite> FastAPI [9, 10, 11]  </cite>

In [5]:
app = FastAPI()

For creating **class-based** views you can use **@cbv** decorator from **fastapi-utils**. <br/>
The motivation of using it: <br/>
```
Stop repeating the same dependencies over and over in the signature of related endpoints.
```
> > -- <cite> Stack Overflow [12]  </cite>

**RealTimeCurrencyConverter** class which will get the real-time exchange rate and convert the currency and return the converted amount. <br/>

<cite>**requests.get(url)** load the page in our python program and then .json() will convert the page into the json file. We store it in a data variable. </cite>


```
class RealTimeCurrencyConverter():
    def __init__(self,url):
            self.data = requests.get(url).json()
            self.currencies = self.data['rates']

    def convert(self, from_currency, to_currency, amount): 
        initial_amount = amount 
        if from_currency != 'USD' : 
            amount = amount / self.currencies[from_currency] 
  
        # limiting the precision to 4 decimal places 
        amount = round(amount * self.currencies[to_currency], 4) 
        return amount
```
> > -- <cite> Stack Overflow [13]  </cite>

In [6]:
url = 'https://api.exchangerate-api.com/v4/latest/USD'
router = InferringRouter()  # Step 1: Create a route

In [7]:
@cbv(router)  # Step 2: Create and decorate a class to hold the endpoints
class RealTimeCurrencyConverter():
    # Step 3: Add dependencies as class attributes
    @router.get("/curr_exchange_csv_api")       
    def __init__(self,url):
            self.data = requests.get(url).json()
            self.currencies = self.data['rates']

    def convert(self, from_currency, to_currency, amount): 
        initial_amount = amount 
        if from_currency != 'USD' : 
            amount = amount / self.currencies[from_currency] 

        # limiting the precision to 4 decimal places 
        amount = round(amount * self.currencies[to_currency], 4) 
        return amount 

In [8]:
app.include_router(router)         

In [9]:
converter = RealTimeCurrencyConverter(url)

In [10]:
exchange_df = preprocessing_data('./objects_data_small_slide.csv', 'Amount', 'Object ID')

In [11]:
new_curr_df = apply_by_row(exchange_df, 'Currency', 'USD', 'Amount', 'std_amount_column')

In [12]:
df_export_to_csv(new_curr_df, './new_curr_exchanged_data.csv')

## References

[1] From https://realpython.com/api-integration-in-python/#rest-architecture <br/>
[2] From https://qatechhub.com/rest-api-introduction/ <br/> 
[3] From https://stevenpcurtis.medium.com/endpoint-vs-api-ee96a91e88ca <br/>
[4] From https://towardsdatascience.com/using-github-pages-for-creating-global-api-76b296c4b3b5 <br/>
[5] From https://blog.hubspot.com/website/api-endpoint <br/>
[6] From https://realpython.com/python-web-applications/ <br/>
[7] From https://anderfernandez.com/blog/como-crear-api-en-python/ <br/>
[8] From https://www.uvicorn.org/ <br/>
[9] From https://fastapi.tiangolo.com/ <br/>
[10] From https://github.com/tiangolo/fastapi <br/>
[11] From https://github.com/tiangolo/fastapi/blob/master/fastapi/applications.py <br/>
[12] From https://stackoverflow.com/questions/63853813/how-to-create-routes-with-fastapi-within-a-class <br/>
[13] From https://data-flair.training/blogs/currency-converter-python/ <br/>
[14] From https://www.roseindia.net/answers/viewqa/pythonquestions/98745-ModuleNotFoundError-No-module-named-fastapi-utils.html <br/>
[15] From https://unix.stackexchange.com/questions/125757/make-complains-missing-separator-did-you-mean-tab <br/>
[16] From https://www.twilio.com/blog/how-run-flask-application <br/>
[17] From https://flask.palletsprojects.com/en/2.0.x/config/  <br/>
[18] From https://flask.palletsprojects.com/en/1.1.x/deploying/#deployment <br/>
[19] From https://realpython.com/python-web-applications/#reader-comments <br/>
